In [1]:
import os
import pickle
import scipy.io
import numpy as np
import numpy.linalg


from scfw.scopt import scopt
from scfw.frank_wolfe import frank_wolfe
import scfw.portfolio as pr
import matplotlib.pyplot as plt

In [2]:
data_folder = '../data'

problem_name = 'syn_1000_800_10_50_3'
data = scipy.io.loadmat(os.path.join(data_folder, problem_name + '.mat'))
R = data['W']
results = {problem_name: {}}

In [3]:
results_folder = './results/'
results_file=os.path.join(results_folder,'portfolio',problem_name+'.pckl')
results = {problem_name: {}}
if os.path.isfile(results_file):
    with open(results_file, "rb") as f:
        try:
            results=pickle.load(f)
        except Exception: # so many things could go wrong, can't be more specific.
            pass 

In [4]:
np.random.seed(42)

## Data preprocessing and parameters initialization

In [5]:
N, n = R.shape
Mf = 2
nu = 3

I = np.eye(n)
Iv = np.ones(n)

#running parameters
x0 = np.ones(n) / n
f0,dot_product0=pr.portfolio(R,x0)
sigma_f=min(np.linalg.eigvalsh(pr.hess_portfolio(R, x0, dot_product0)))
if sigma_f<0:
    sigma_f=1e-10
terminate_tol = 1e-15

#parameters for FW
FW_params={
    'iter_FW':50000,
    'line_search_tol':1e-10,
    'rho':np.sqrt(n), #parameters for ll00
    'diam_X':np.sqrt(2),
    'sigma_f':sigma_f,                   
}


sc_params={
    #parameters for SCOPT
    'iter_SC': 1000,
    'Lest': 'backtracking',#,'estimate', #estimate L
    'use_two_phase':False,
    #FISTA parameters
    'fista_type': 'mfista',
    'fista_tol': 1e-5,
    'fista_iter': 1000,
    #Conjugate Gradient Parameters
    'conj_grad_tol':1e-5,
    'conj_grad_iter':1000,
}

## Auxilary functions

In [6]:
func_x = lambda x: pr.portfolio(R,x)
func_beta = lambda x, s, beta, dot_product, dot_product_s:lr.log_reg(R, (1 - beta) * x + beta * s, (dot_product)*(1-beta)+(dot_product_s)*beta,gamma)
func_beta = lambda x, s, beta, dot_product, dot_product_s: pr.portfolio(R,(1 - beta) * x + beta * s,(1 - beta) * dot_product + beta * dot_product_s)
grad_x = lambda x, dot_product: pr.grad_portfolio(R, x, dot_product)
grad_beta = lambda x, s, beta, dot_product, dot_product_s: pr.grad_portfolio(R, (1 - beta) * x + beta * s, (1 - beta) * dot_product + beta * dot_product_s)
hess_x = lambda x, dot_product: pr.hess_portfolio(R, x, dot_product)
hess_mult_x = lambda x, dot_product: pr.hess_mult_portfolio(R, x, dot_product)
hess_mult_vec_x = lambda s, dot_product: pr.hess_mult_vec(R, s, dot_product)
extra_func = lambda x: R @ x
linear_oracle = lambda grad: pr.linear_oracle_simplex(grad)
llo_oracle = lambda x, r, grad, rho: pr.llo_oracle(x, r, grad,rho)
prox_func = lambda x, L: pr.proj_simplex(x)

In [7]:
points = [x0]
 
for _ in range(1):
    x = np.abs(np.random.rand(len(x0)))
    x = x / sum(x)
    points.append(x)

## Run FW

In [8]:
run_alpha_policies = ['new_lloo']
x0=points[0]

for policy in run_alpha_policies:
    x, alpha_hist, Gap_hist, Q_hist, time_hist  = frank_wolfe(func_x,
                       func_beta,                                       
                       grad_x,
                       grad_beta,
                       hess_mult_x,
                       extra_func,                                                    
                       Mf,
                       nu,
                       linear_oracle,                                                    
                       x0,
                       FW_params,
                       hess=hess_x, 
                       lloo_oracle=llo_oracle,                                                 
                       alpha_policy=policy,                                                    
                       eps=terminate_tol, 
                       print_every=1000, 
                       debug_info=False)
    
    results[problem_name][policy] = {
        'x': x,
        'alpha_hist': alpha_hist,
        'Gap_hist': Gap_hist,
        'Q_hist': Q_hist,
        'time_hist': time_hist,
    }
    
    with open(results_file, 'wb') as f:
        pickle.dump(results, f)

********* Algorithm starts *********
iter = 1, stepsize = 0.380964922288197, criterion = 1e-05, upper_bound=0.02128931286912572, lower_bound=-4.917163298675437, real_Gap=4.938452611544562
iter = 1000, stepsize = 0.0021676977600549713, criterion = 1e-05, upper_bound=-4.4750868954505405, lower_bound=-4.4754663579290535, real_Gap=0.00037946247851294856
iter = 2000, stepsize = 0.0010774499935317546, criterion = 1e-05, upper_bound=-4.475091411295941, lower_bound=-4.475360690114719, real_Gap=0.0002692788187781403
iter = 3000, stepsize = 0.0005360074110300544, criterion = 1e-05, upper_bound=-4.475092312124611, lower_bound=-4.475206687859492, real_Gap=0.00011437573488137787
iter = 4000, stepsize = 0.0006786638103880968, criterion = 1e-05, upper_bound=-4.475092578701338, lower_bound=-4.475161664911761, real_Gap=6.908621042356344e-05
iter = 5000, stepsize = 0.00025533030304746734, criterion = 1e-05, upper_bound=-4.475092716476, lower_bound=-4.47515282749889, real_Gap=6.011102288994152e-05
iter =

In [9]:
run_alpha_policies = []#["backtracking","standard","line_search","icml"]

for policy in run_alpha_policies:
    print(policy)
    #policy_results = []
    #for i in range(len(points)):
    #    x0 = points[i]
    x, alpha_hist, Gap_hist, Q_hist, time_hist = frank_wolfe(func_x,
                        func_beta,                                       
                        grad_x,
                        grad_beta,
                        hess_mult_x,
                        extra_func,                                                    
                        Mf,
                        nu,
                        linear_oracle,                                                    
                        x0,
                        FW_params,                                       
                        alpha_policy=policy,                                                    
                        eps=terminate_tol,
                        print_every=1000, 
                        debug_info=False)
        
    results[problem_name][policy]={
    'x': x,
    'alpha_hist': alpha_hist,        
    'Q_hist': Q_hist,
    'Gap_hist': Gap_hist,        
    'time_hist': time_hist,}
    with open(results_file, 'wb') as f:
        pickle.dump(results, f)
#    results[problem_name][policy] = policy_results

## Run SCOPT

In [10]:
x, alpha_hist, Q_hist, time_hist = scopt(func_x,
        grad_x,
        hess_mult_x,
        hess_mult_vec_x,
        Mf,
        nu,
        prox_func,
        x0,  
        sc_params,                                              
        eps=terminate_tol,                                              
        print_every=1)
    
results[problem_name]['scopt'] = {
    'x': x,
    'alpha_hist': alpha_hist,
#    'Gap_hist': Gap_hist,
    'Q_hist': Q_hist,
    'time_hist': time_hist,
#    'grad_hist': grad_hist
}

iter =    1, stepsize = 5.663e-01, rdiff = 4.796e-01 , f = 0.0212893

Fista err = 9.137e-06; Subiter =  32; subproblem converged!

iter =    2, stepsize = 7.532e-01, rdiff = 2.064e-01 , f = -2.59625

Fista err = 9.999e-06; Subiter = 210; subproblem converged!

iter =    3, stepsize = 9.231e-01, rdiff = 5.309e-02 , f = -4.02128

Fista err = 9.818e-06; Subiter =  18; subproblem converged!

iter =    4, stepsize = 9.937e-01, rdiff = 4.036e-03 , f = -4.44046

Fista err = 3.975e-11; Subiter =   2; subproblem converged!

iter =    5, stepsize = 1.000e+00, rdiff = 2.237e-05 , f = -4.47488

Fista err = 2.515e-12; Subiter =   2; subproblem converged!

boo
Convergence achieved!
iter =    6, stepsize = 0.000e+00, rdiff = 0.000e+00,value=-4.47509

16.547945976257324


In [11]:
with open(results_file, 'wb') as f:
    pickle.dump(results, f)